In [2]:
# !pip install bs4
# !pip install selenium

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=fbff3116dbe7c26f878bd2049dd9682b8827ed8d732c2fe59f25f67d550b505d
  Stored in directory: c:\users\cepl\appdata\local\pip\cache\wheels\e4\62\1d\d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4
     ---------------------------------------- 6.6/6.6 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00


In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import UnexpectedAlertPresentException
import numpy as np
import pandas as pd

In [7]:
# Setting Chrome options
options = Options()
options.add_argument("--disable-notifications")

# Defining the URL of the main page
base_url = "https://www.instahyre.com/search-jobs/" 

# Defining the class name of the HTML elements to scrape
class_name = 'employer-block'

# Defining the number of times to repeat the scraping process
num_repeats = 50


In [1]:
driver = webdriver.Chrome(options=options)

# Maximizing browser window size
driver.maximize_window()

# Making a GET request to the URL
driver.get(base_url)

job_data = []

# Finding data on webpage
for i in range(1,71):
    print("Page", i)

    driver.implicitly_wait(10)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # Finding all HTML elements with the specified class name
    cards = soup.find_all(class_=class_name)
    
    # Looping through each card and opening its link to get its contents
    for card in cards:
        link = card.find('a')['href']

        print("Navigating to:", link)
        content_driver = webdriver.Chrome(options=options)
        content_driver.get(link)

        try:
            alert = content_driver.switch_to.alert
            alert.accept()
        except:
            pass

        content_html = content_driver.page_source
        content_soup = BeautifulSoup(content_html, 'html.parser')

        # Waiting for page to load
        content_driver.implicitly_wait(10)
        
        job_id          = int(link.split('-')[1])
        job_title       = content_soup.find("h1").text.strip()
        company_name    = content_soup.find(class_="company-name").text.strip()
        experience      = content_soup.find(class_="experience").text.strip()
        job_locations   = content_soup.find(class_="job-locations").text.strip().replace(experience, "")
        HR_name         = content_soup.find(class_="rec-name").text.strip()
        founded_in_elem = content_soup.find("div", {"ng-if": "employer.company_founded"})
        employees_count = content_soup.find("div", {"ng-if": "employer.employee_count"})
        employees_count = employees_count.text.strip() if employees_count and employees_count.text else employees_count
        skills_children = content_soup.find(class_="tags job-description-skills")
        skills_children = skills_children.children if skills_children and skills_children.children else [].append(skills_children)

        skills_list     = []
        
        if skills_children is not None:
            for skill in skills_children:
                skill = skill.text.strip()
                if (skill and skill != "\\n"):
                    skills_list.append(skill)
        skills= "N/A" if skills_list is None      else ", ".join(skills_list)
        founded_in= "N/A" if founded_in_elem is None  else founded_in_elem.text.strip()
        job_data.append({
            "company_name"      : company_name,
            "job_title"         : job_title,
            "job_locations"     : job_locations,
            "experience"        : experience,
            "hr_name"           : HR_name,
            "founded_in"        : founded_in,
            "employees_count"   : employees_count,
            "skills"            : skills,
            "job_id"            : job_id
        })

        # Returning to the previous page
        content_driver.back()
        content_driver.quit()
    
    driver.implicitly_wait(20)
        
    # Clicking the button to go to the next page
    next_button = driver.find_element(By.CSS_SELECTOR, '[ng-click="nextPage()"]')

    if next_button is not None:
        next_button.click()
    else:
        print("No next page button found.")


In [14]:
#Converting to DataFrame

df=pd.DataFrame(job_data)

In [15]:
df

,company_name,job_title,job_locations,experience,hr_name,founded_in,employees_count,skills,job_id
0,Cogoport,Service Ops (Drive),Gurgaon,0-2 Years,Revati Agnihotri,Founded in 2016,200 - 500 employees,"Data Entry, Documentation, Operations Management",249688
1,Hubilo,Marketing Intern (Drive),Work From Home,0-1 Years,Tejashwini Kanashetti,Founded in 2015,50 - 200 employees,"Digital Marketing, Marketing Communication",252261
2,RingCentral,Sr. Revenue Analyst (Drive),Bangalore,2-4 Years,Azhar Ahmed,Founded in 2003,More than 1000 employees,"Accounting, Chartered Accountant, ERP Implemen...",251583
3,,,,- Years,Marsina Waffa,N/A,None,,249768
4,,,,- Years,Marsina Waffa,N/A,None,,250986
...,...,...,...,...,...,...,...,...,...
333,Inspire Brands,Software Engineer,Hyderabad,2-4 Years,Palak Parashar,Founded in 2018,More than 1000 employees,"Java, Spring Boot, Microservices",252665
334,LTIMindtree,SAP FICO Consultant,Hyderabad,7-12 Years,Md Naveed,Founded in 1996,More than 1000 employees,"SAP FICO, SAP ERP, Taxation, Finance, SAP Modu...",252663
335,Macgence,Business Development Manager,Noida,1-3 Years,Priyanka U,Founded in 2020,10 - 50 employees,"Sales, Customer Service, Media Relations, CRM,...",252662
336,Macgence,Presales Executive,Noida,3-5 Years,Priyanka U,Founded in 2020,10 - 50 employees,"Sales, Telecalling, Transcription, Marketing, ...",252667
